<a href="https://colab.research.google.com/github/aditya123456/automation-framework-modular/blob/main/Deploying_a_Streamlit_App_with_Gemini_Pro_on_Cloud_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying a Streamlit App with Gemini Pro on Cloud Run

Lab link - # https://bit.ly/pune-genai-lab
---


# Overview
This Colab notebook guides you through integrating the Vertex AI Gemini API with applications and deploying a Streamlit application on Google Cloud Run.

Gemini Pro, part of Google DeepMind's generative AI models, supports natural language tasks, multiturn text, code chat, and code generation, alongside multimodal prompts with its Vision model.


# Gemini
Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro Vision and Gemini Pro models.


# Vertex AI Gemini API
The Vertex AI Gemini API provides a unified interface for interacting with Gemini models. There are currently two models available in the Gemini API:

**Gemini Pro model** (`gemini-pro`): Designed to handle natural language tasks, multiturn text and code chat, and code generation.


**Gemini Pro Vision model** (`gemini-pro-vision`): Supports multimodal prompts. You can include text, images, and video in your prompt requests and get text or code responses.



You can interact with the Gemini API using the following methods:


*   Use the [Vertex AI Studio](https://cloud.google.com/generative-ai-studio?hl=en) for quick testing and command generation
*   Use cURL commands
*   Use the Vertex AI SDK

This lab focuses on using the **Vertex AI SDK for Python** to call the Vertex AI Gemini API.


For more information, see the [Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) documentation.

---



# Objectives

*   Understand the integration of the Vertex AI Gemini API into a [Streamlit](https://streamlit.io/) application.
*   Build and deploy the application on Google Cloud Run.

---

# Prerequisites
Before starting, ensure you have access to a Google Cloud account and the `gcloud` CLI is configured for your project.

To smoothly proceed with deploying a Streamlit app integrated with Gemini Pro on Google Cloud Run, complete the following prerequisite step:

*   **Google Cloud Project:** Have a Google Cloud project created and selected in your Google Cloud Console. If you haven't done this, visit the [Google Cloud Console](https://console.cloud.google.com/) to create a new project.


*  **Billing Enabled:** Ensure your Google Cloud project has billing enabled. Visit the [Billing section](https://console.cloud.google.com/billing) of the Google Cloud Console to verify or set up your billing account.


Follow the presentation in this link to enable billing for today's workshop - [Enable billing](https://bit.ly/genai-demo-pune)


---

# Setup
Before starting, ensure you have access to a Google Cloud account and the `gcloud` CLI is configured for your project.

# Step 1: Clone the Repository
Clone the repository containing the sample Streamlit application integrated with the Gemini Pro API.

In [ ]:
#!git clone https://github.com/GoogleCloudPlatform/generative-ai.git
!git clone https://github.com/jitu028/generative-ai.git
%ls

# Step 2: Set Up Virtual Environment and Dependencies
Create a Python virtual environment and install the required dependencies.

In [ ]:
!apt install python3.10-venv -q
!python3 -m venv gemini-streamlit
!source gemini-streamlit/bin/activate

# Step 3: Set Environment Variables
Configure the required environment variables for your Google Cloud project and region.

In [ ]:
import os

# Replace 'your-project-id' with your Google Cloud project ID

#os.environ['GCP_PROJECT'] = 'your-project-id'
os.environ['GCP_PROJECT'] = 'gcp-demo-028'
os.environ['GCP_REGION'] = 'us-central1'


# Step 4: Install Required Tools
Ensure you have the gcloud and gsutil command-line tools installed and authenticated. This setup is crucial for deploying applications on Google Cloud Platform.

In [ ]:
#@title Authenticate with Google Cloud and your project ID

from google.colab import auth

gcp_project_id = 'gcp-demo-028' # @param {type: "string"} # replace 'you-project-id' with your project id

auth.authenticate_user(project_id=gcp_project_id)

# Install Google Cloud SDK (Skip if already installed)
!curl https://sdk.cloud.google.com | bash

# Initialize gcloud (Skip if already configured)
!gcloud init

#Enable required APIs - Cloud Run, Artifact Registry, and Cloud Build APIs for your project by running the following command, replace "your-project-id" with your actual project ID:
!gcloud services enable run.googleapis.com artifactregistry.googleapis.com cloudbuild.googleapis.com --project=$gcp_project_id

# Step 5: Run the Application Locally
Start the Streamlit application locally to test its functionality.

In [ ]:
%ls

%cd generative-ai/gemini/sample-apps/gemini-streamlit-cloudrun
!pip install -r requirements.txt

!streamlit run app.py \
  --browser.serverAddress=localhost \
  --browser.gatherUsageStats=false  \
  --server.enableCORS=false \
  --server.enableXsrfProtection=false \
  --server.port 8501

# Step 6: Build and Deploy the Application to Cloud Run
Now, let's deploy the application on Cloud Run.

this steps take 4-5 minutes to complete

In [ ]:
# Enable the APIs

!gcloud services enable run.googleapis.com \
                          artifactregistry.googleapis.com \
                          cloudbuild.googleapis.com \
                          aiplatform.googleapis.com \
                          --project="$GCP_PROJECT"

!AR_REPO='gemini-repo' # artifact repo name

!SERVICE_NAME='gemini-streamlit-app' # service account name

# create repo in artifacts registry
!gcloud artifacts repositories create gemini-repo --location=us-central1 --repository-format=Docker

# build and push th
!gcloud builds submit --tag "us-central1-docker.pkg.dev/$GCP_PROJECT/gemini-repo/gemini-streamlit-app"

In [ ]:
# deploy to cloud run

!gcloud run deploy gemini-streamlit-app \
  --port=8080 \
  --image="us-central1-docker.pkg.dev/$GCP_PROJECT/gemini-repo/gemini-streamlit-app" \
  --allow-unauthenticated \
  --region=us-central1 \
  --platform=managed  \
  --project="$GCP_PROJECT" \
  --set-env-vars=GCP_PROJECT="$GCP_PROJECT",GCP_REGION="$GCP_REGION"


# Conclusion

Upon successful deployment, you'll receive a URL to access the Cloud Run application.

This application demonstrates how to integrate and utilize the Vertex AI Gemini API within a Streamlit application, showcasing Google Cloud's capabilities for hosting AI-powered applications.

Remember to replace placeholders like **'your-project-id'** and **'your-region'** with actual values from your Google Cloud project.

**Clean Up Step**  
# Clean up the resoruces

In [ ]:

#Remember to replace placeholders like 'your-project-id' and 'your-region' with actual values from your Google Cloud project.

# Delete a Cloud Run service
!gcloud run services list

# Delete a Cloud Run service
#!gcloud run services delete [SERVICE-NAME] --region [REGION] --platform managed
!gcloud run services delete gemini-streamlit-app --region us-central1 --platform managed


# List images in Artifact registry
!gcloud artifacts repositories list

# List images in a repository
!gcloud artifacts docker images list us-central1-docker.pkg.dev/$GCP_PROJECT/gemini-repo

# Delete an image
!gcloud artifacts docker images delete us-central1-docker.pkg.dev/$GCP_PROJECT/gemini-repo/gemini-streamlit-app

# List images in a repository
#!gcloud artifacts docker images list us-central1-docker.pkg.dev/your-project-id/gemini-repo

# Delete an Artifact Registry repository
!gcloud artifacts repositories delete gemini-repo --location=us-central1 --quiet

#disable APIs - Cloud Run, Artifact Registry, and Cloud Build APIs for your project by running the following command:
!gcloud services disable run.googleapis.com artifactregistry.googleapis.com cloudbuild.googleapis.com --project=$GCP_PROJECT